# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure and see screenshot in images folder
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

290

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 75) & (weather_data["Max Temp"] <= 90)].dropna()

narrowed_city_df= weather_data
narrowed_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,ostrovnoy,RU,68.05,39.51,
1,mahebourg,MU,-20.41,57.70,
2,qaanaaq,GL,77.48,-69.36,
3,zhuhai,CN,40.71,112.04,
4,cape town,ZA,-33.93,18.42,
...,...,...,...,...,...
543,yabrud,SY,33.97,36.66,
544,paraiso,MX,24.01,-104.61,
545,veraval,FR,49.65,0.71,
546,novyy urgal,RU,51.07,132.56,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [ ]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 0: ostrovnoy.
Missing field/result... skipping.
------------
Retrieving Results for Index 1: mahebourg.
Closest hotel in mahebourg is Le Peninsula Bay Beach Resort & Spa.
------------
Retrieving Results for Index 2: qaanaaq.
Closest hotel in qaanaaq is Qaanaaq Hotel.
------------
Retrieving Results for Index 3: zhuhai.
Missing field/result... skipping.
------------
Retrieving Results for Index 4: cape town.
Closest hotel in cape town is President Hotel.
------------
Retrieving Results for Index 5: atuona.
Closest hotel in atuona is Hôtel Hiva Oa Hanakee Lodge.
------------
Retrieving Results for Index 6: flinders.
Closest hotel in flinders is Lakeview Hotel Motel.
------------
Retrieving Results for Index 7: opuwo.
Closest hotel in opuwo is Opuwo Country Lodge.
------------
Retrieving Results for Index 8: tiksi.
Closest hotel in tiksi is Umka.
------------
Retrieving Results for Index 9: pushkinskiye gory.
Closest hotel in pushkinskiye gory is Baza Otdykha 

Retrieving Results for Index 78: crotone.
Closest hotel in crotone is Hotel San Giorgio.
------------
Retrieving Results for Index 79: baglung.
Closest hotel in baglung is Hotel Peace Palace, Baglung, Nepal.
------------
Retrieving Results for Index 80: raglan.
Closest hotel in raglan is Raglan Harbour View Hotel.
------------
Retrieving Results for Index 81: jamestown.
Closest hotel in jamestown is Jamestown Hotel.
------------
Retrieving Results for Index 82: sur.
Closest hotel in sur is Sur Plaza Hotel.
------------
Retrieving Results for Index 83: araouane.
Missing field/result... skipping.
------------
Retrieving Results for Index 84: emerald.
Closest hotel in emerald is Emerald Maraboon Motor Inn.
------------
Retrieving Results for Index 85: launceston.
Closest hotel in launceston is Hotel Grand Chancellor Launceston.
------------
Retrieving Results for Index 86: nikolskoye.
Missing field/result... skipping.
------------
Retrieving Results for Index 87: ambilobe.
Closest hotel i

Closest hotel in bjornevatn is Snowhotel Kirkenes & Gamme Northern Lights Cabins.
------------
Retrieving Results for Index 156: narsaq.
Closest hotel in narsaq is Igdlo Guesthouse.
------------
Retrieving Results for Index 157: charlestown.
Missing field/result... skipping.
------------
Retrieving Results for Index 158: trat.
Closest hotel in trat is Avada Hotel.
------------
Retrieving Results for Index 159: manado.
Closest hotel in manado is Four Points by Sheraton Manado.
------------
Retrieving Results for Index 160: trapani.
Closest hotel in trapani is Hotel Vittoria.
------------
Retrieving Results for Index 161: sao joao da barra.
Closest hotel in sao joao da barra is Pousada Rio Sol.
------------
Retrieving Results for Index 162: saint-philippe.
Missing field/result... skipping.
------------
Retrieving Results for Index 163: haines junction.
Closest hotel in haines junction is Stardust Motel.
------------
Retrieving Results for Index 164: kathmandu.
Closest hotel in kathmandu 

Retrieving Results for Index 233: san juan del sur.
Closest hotel in san juan del sur is Hotel Victoriano.
------------
Retrieving Results for Index 234: katobu.
Closest hotel in katobu is La Ode Husein.
------------
Retrieving Results for Index 235: banjar.
Closest hotel in banjar is Banjar Indah Hotel.
------------
Retrieving Results for Index 236: provideniya.
Missing field/result... skipping.
------------
Retrieving Results for Index 237: andradas.
Closest hotel in andradas is Hotel Dela Inn.
------------
Retrieving Results for Index 238: bose.
Closest hotel in bose is Hotel Royal.
------------
Retrieving Results for Index 239: manzhouli.
Closest hotel in manzhouli is Manzhouli Shangri-La Hotel.
------------
Retrieving Results for Index 240: lockhart.
Closest hotel in lockhart is Commercial Hotel.
------------
Retrieving Results for Index 241: anadyr.
Closest hotel in anadyr is Chukotka hotel.
------------
Retrieving Results for Index 242: bandarbeyla.
Closest hotel in bandarbeyla 

Retrieving Results for Index 310: airai.
Missing field/result... skipping.
------------
Retrieving Results for Index 311: balkanabat.
Closest hotel in balkanabat is serdarlan oyi.
------------
Retrieving Results for Index 312: syumsi.
Closest hotel in syumsi is Gostinitsa "Polar".
------------
Retrieving Results for Index 313: loukhi.
Closest hotel in loukhi is Remstroyrekonstruktsiya.
------------
Retrieving Results for Index 314: baruun-urt.
Closest hotel in baruun-urt is "Сод" Буудал.
------------
Retrieving Results for Index 315: miraflores.
Closest hotel in miraflores is hotel playa alta.
------------
Retrieving Results for Index 316: gunjur.
Closest hotel in gunjur is Nemasu Ecolodge.
------------
Retrieving Results for Index 317: wajima.
Closest hotel in wajima is Wajiima Onsen Yashio.
------------
Retrieving Results for Index 318: buala.
Closest hotel in buala is Ghaseali Island Resort.
------------
Retrieving Results for Index 319: acari.
Closest hotel in acari is Pousada Garg

In [ ]:
# Display the hotel dataframe
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)

# Display Map and see screenshot in images forder
fig